In [1]:
from scipy.stats import chisquare,chi2_contingency
import pandas as pd
import numpy as np

#### Chi2_contingency test tells about the dependence of the variables. But it doesn't talk about the depth of the relation. 
##### CramerV method is implemented using contigency table and chisquare value by finding phi squared(which is essential the normalised value of chi2)
##### Steps 
1. Compute the contigency table (sometimes known as confusion matrix)
2. Calulate the Chi-squared value
3. Find the total number of records (n)
4. Find the phi-squared value by chi2/n
5. Get the shape of the confusion matrix r,k=confusion_matrix.shape
6. Correct for bais (Bais correction) by phi2_cor=max(0,phi2-((r-1)*(k-1))/n-1)
7. Apply corrections for r,k rcor=r-(r-1)**2/n-1; kcor=k-(k-1)**2/n-1
8. Compute the correlations using R=np.sqrt(phi2/(rcor-1)*(kcor-1))


In [29]:
df=pd.read_csv(r"/Users/karthikeya/Downloads/rgmx-ps-vtm (1)/data/02_intermediate/ps/attributes.csv")
df.head()

,DESCRIPTION,Branded/Private Label,Manufacturer,Range/Trading Company,Brand,Number in a Pack,Active_Ingredient,MANUFACTURER_NM,BRAND_NM,SUBBRAND_NM,...,PARACETAMOL_AO,IBUPROFEN_AO,PRICE_TIER,Brand.1,BRAND_NM.1,Range/Trading Company_NEW,Unnamed: 24,Price,PRICE_TIER_MANUAL_MAPPED,sales
0,CALPOL S/F SUSP 100ML,Branded,Johnson & Johnson Ltd,Calpol,Calpol Analgesics,Not Applicable,Paracetamol,MCNEIL,CALPOL,CALPOL INFANT SUSPENSION,...,PARACETAMOL,AO,Premium,Calpol Analgesics,CALPOL,Johnson & Johnson Ltd,NaN,0.190383,Mainstream,6524028.0
1,ANADIN EX ASP/PRC CPLTS 16PK,Branded,Haleon,Anadin,Anadin Analgesics,16 in a Pack,Compounds,HALEON,ANADIN,ANADIN EXTRA,...,AO,AO,Value,Anadin Analgesics,ANADIN,Haleon,NaN,0.125147,Value,2265611.0
2,CALPOL SIX PLUS S/F 80ML,Branded,Johnson & Johnson Ltd,Calpol,Calpol Analgesics,Not Applicable,Paracetamol,MCNEIL,CALPOL,CALPOL SIXPLUS,...,PARACETAMOL,AO,Premium,Calpol Analgesics,CALPOL,Johnson & Johnson Ltd,NaN,0.190383,Mainstream,4669969.0
3,NRFN CHLD S/F C/F SUSP 100ML,Branded,Reckitt Benckiser,Nurofen,Nurofen Child Analgesics,Not Applicable,Ibuprofen,RECKITT BENCKISER,NUROFEN,NUROFEN FOR CHILDREN,...,AO,IBUPROFEN,Premium,Nurofen Child Analgesics,NUROFEN,Reckitt Benckiser,NaN,0.200080,Mainstream,3241569.0
4,TESCO PRCTML CPLTS 16PK,Private Label,Tesco Food Stores Ltd,Tesco,Tesco Analgesics,16 in a Pack,Paracetamol,PRIVATE LABEL,PRIVATE LABEL,PRIVATE LABEL,...,PARACETAMOL,AO,Value,Tesco Analgesics,PRIVATE LABEL,PL,NaN,0.036136,Value,12497440.0


In [30]:
df.columns

Index(['DESCRIPTION', 'Branded/Private Label', 'Manufacturer',
       'Range/Trading Company', 'Brand', 'Number in a Pack',
       'Active_Ingredient', 'MANUFACTURER_NM', 'BRAND_NM', 'SUBBRAND_NM',
       'PRODUCT_CATEGORY', 'PRODUCT_SUB_CATEGORY', 'GSL_PHARMACY',
       'CHILD_ADULT', 'PL_AO', 'FORM_TYPE', 'PACK_SIZE', 'DRUG_SUBSTANCE',
       'PARACETAMOL_AO', 'IBUPROFEN_AO', 'PRICE_TIER', 'Brand.1', 'BRAND_NM.1',
       'Range/Trading Company_NEW', 'Unnamed: 24', 'Price',
       'PRICE_TIER_MANUAL_MAPPED', 'sales'],
      dtype='object')

In [31]:
#filtering the data
df=df.select_dtypes(include='object')
df=df[['Branded/Private Label','MANUFACTURER_NM','BRAND_NM','CHILD_ADULT','PRICE_TIER','FORM_TYPE','SUBBRAND_NM','PARACETAMOL_AO','IBUPROFEN_AO']]
df.head()

,Branded/Private Label,MANUFACTURER_NM,BRAND_NM,CHILD_ADULT,PRICE_TIER,FORM_TYPE,SUBBRAND_NM,PARACETAMOL_AO,IBUPROFEN_AO
0,Branded,MCNEIL,CALPOL,CHILD,Premium,LIQUID,CALPOL INFANT SUSPENSION,PARACETAMOL,AO
1,Branded,HALEON,ANADIN,ADULT,Value,CAPLETS,ANADIN EXTRA,AO,AO
2,Branded,MCNEIL,CALPOL,CHILD,Premium,LIQUID,CALPOL SIXPLUS,PARACETAMOL,AO
3,Branded,RECKITT BENCKISER,NUROFEN,CHILD,Premium,LIQUID,NUROFEN FOR CHILDREN,AO,IBUPROFEN
4,Private Label,PRIVATE LABEL,PRIVATE LABEL,ADULT,Value,TABLETS,PRIVATE LABEL,PARACETAMOL,AO


In [32]:
def cramerv(cm):
    n=cm.sum().sum()
    r,k=cm.shape
    chi2=chi2_contingency(cm)[0]
    phi2=chi2/n
    phi2_cor=max(0,phi2-((r-1)*(k-1))/n-1)
    r_cor=r-((r-1)**2/n-1)
    k_cor=k-((k-1)**2/n-1)
    if min(r_cor-1,k_cor-1)==0:
        return 0
    else:
        correlation=np.sqrt(phi2_cor/min(r_cor-1,k_cor-1))
        return correlation
    
def confusion_matrix(df):
    n=len(df.columns)
    confusion_matrix=pd.DataFrame(np.zeros((n,n)),index=df.columns,columns=df.columns)
    
    #calculate corelation for each cell
    for col1 in df.columns:
        for col2 in df.columns:
            cm=pd.crosstab(df[col1],df[col2])
            if col1==col2:
                confusion_matrix.loc[col1,col2]=1.0
            else:
                confusion_matrix.loc[col1,col2]=cramerv(cm)
    return confusion_matrix
                

In [33]:
confusion_matrix(df)

,Branded/Private Label,MANUFACTURER_NM,BRAND_NM,CHILD_ADULT,PRICE_TIER,FORM_TYPE,SUBBRAND_NM,PARACETAMOL_AO,IBUPROFEN_AO
Branded/Private Label,1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
MANUFACTURER_NM,0.0,1.000000,0.888176,0.0,0.0,0.0,0.801392,0.0,0.0
BRAND_NM,0.0,0.888176,1.000000,0.0,0.0,0.0,0.843461,0.0,0.0
CHILD_ADULT,0.0,0.000000,0.000000,1.0,0.0,0.0,0.000000,0.0,0.0
PRICE_TIER,0.0,0.000000,0.000000,0.0,1.0,0.0,0.000000,0.0,0.0
FORM_TYPE,0.0,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.0,0.0
SUBBRAND_NM,0.0,0.801392,0.843461,0.0,0.0,0.0,1.000000,0.0,0.0
PARACETAMOL_AO,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0
IBUPROFEN_AO,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,1.0
